In [1]:
import cv2
import os
import numpy as np
from glob import glob

# Define paths
base_image_paths = [
    r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\coral-reef-1.jpg",
    r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\coral-reef-2.jpg"
]
photos_folder_path = r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\photos"
output_folder = r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\output"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load all images from the folder with common extensions
photo_paths = []
for ext in ["*.jpg", "*.jpeg", "*.png"]:
    photo_paths.extend(glob(os.path.join(photos_folder_path, ext)))

# Check if there are images to process
num_photos = len(photo_paths)
if num_photos == 0:
    print(f"No images found in the folder '{photos_folder_path}'. Please check the path and try again.")
else:
    print(f"Found {num_photos} images in the folder '{photos_folder_path}'.")

    def split_image_into_regions(image, rows, cols):
        """Split the base image into regions for overlay."""
        h, w = image.shape[:2]
        region_height = h // rows
        region_width = w // cols
        regions = []
        
        for i in range(rows):
            for j in range(cols):
                x_start = j * region_width
                y_start = i * region_height
                x_end = x_start + region_width
                y_end = y_start + region_height
                regions.append((y_start, y_end, x_start, x_end))
        
        return regions

    def overlay_images_on_base(base_image_path, photo_paths, rows, cols):
        """Overlay images onto the base image using a specified grid size."""
        base_image = cv2.imread(base_image_path)
        if base_image is None:
            raise FileNotFoundError(f"Base image not found at {base_image_path}")
        
        regions = split_image_into_regions(base_image, rows, cols)
        output_image = base_image.copy()

        for idx, (y_start, y_end, x_start, x_end) in enumerate(regions):
            if idx >= len(photo_paths):
                break  # Stop if we run out of photos to overlay

            # Load and resize the photo to fit the region
            photo = cv2.imread(photo_paths[idx])
            if photo is None:
                print(f"Warning: Could not read image {photo_paths[idx]}. Skipping.")
                continue

            # Resize the image to fit exactly within the region
            region_height = y_end - y_start
            region_width = x_end - x_start
            photo_resized = cv2.resize(photo, (region_width, region_height), interpolation=cv2.INTER_AREA)

            # Place the resized image onto the corresponding region of the base image
            output_image[y_start:y_end, x_start:x_end] = photo_resized

        return output_image

    # Calculate an optimal grid size based on the number of photos
    grid_size = int(np.ceil(np.sqrt(num_photos)))
    rows = cols = grid_size  # Using a square grid for simplicity

    # Process each base image
    for base_image_path in base_image_paths:
        base_image_name = os.path.basename(base_image_path).split('.')[0]
        
        # Overlay the photos onto the base image
        output_image = overlay_images_on_base(base_image_path, photo_paths, rows, cols)
        
        # Save the output image
        output_path = os.path.join(output_folder, f"{base_image_name}-with-photos.jpg")
        cv2.imwrite(output_path, output_image)
        print(f"Saved output to: {output_path}")


Found 11 images in the folder 'D:\Vortex\ROV\Phase 2\Tasks2025\task 4\photos'.
Saved output to: D:\Vortex\ROV\Phase 2\Tasks2025\task 4\output\coral-reef-1-with-photos.jpg
Saved output to: D:\Vortex\ROV\Phase 2\Tasks2025\task 4\output\coral-reef-2-with-photos.jpg


# Another solution

In [2]:
import cv2
import os
import numpy as np
from glob import glob

# Define paths
base_image_paths = [
    r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\coral-reef-1.jpg",
    r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\coral-reef-2.jpg"
]
photos_folder_path = r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\photos"
output_folder = r"D:\Vortex\ROV\Phase 2\Tasks2025\task 4\output"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load all images from the folder with common extensions
photo_paths = []
for ext in ["*.jpg", "*.jpeg", "*.png"]:
    photo_paths.extend(glob(os.path.join(photos_folder_path, ext)))

# Check if there are images to process
num_photos = len(photo_paths)
if num_photos == 0:
    print(f"No images found in the folder '{photos_folder_path}'. Please check the path and try again.")
else:
    print(f"Found {num_photos} images in the folder '{photos_folder_path}'.")

def detect_grid_cells(image):
    """Detect grid cells in an image using edge detection and contour detection."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours of the grid
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter out small contours and sort contours
    cell_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]
    cell_contours = sorted(cell_contours, key=lambda cnt: (cv2.boundingRect(cnt)[1], cv2.boundingRect(cnt)[0]))
    
    # Calculate bounding boxes for each cell
    cells = [cv2.boundingRect(cnt) for cnt in cell_contours]
    
    return cells

def overlay_images_on_grid(base_image_path, photo_paths, cells):
    """Overlay images onto grid cells detected on the base image, repeating images if necessary."""
    base_image = cv2.imread(base_image_path)
    if base_image is None:
        raise FileNotFoundError(f"Base image not found at {base_image_path}")
    
    output_image = base_image.copy()
    num_cells = len(cells)
    
    # Repeat images if there are fewer images than grid cells
    photos_to_use = (photo_paths * (num_cells // len(photo_paths) + 1))[:num_cells]

    for idx, (x, y, w, h) in enumerate(cells):
        # Load and resize the photo to fit the cell dimensions
        photo = cv2.imread(photos_to_use[idx])
        if photo is None:
            print(f"Warning: Could not read image {photos_to_use[idx]}. Skipping.")
            continue

        # Resize the image to fit within the cell, keeping aspect ratio
        photo_aspect_ratio = photo.shape[1] / photo.shape[0]
        cell_aspect_ratio = w / h

        # Adjust dimensions based on aspect ratio
        if photo_aspect_ratio > cell_aspect_ratio:
            # Fit to width
            new_width = w
            new_height = int(w / photo_aspect_ratio)
        else:
            # Fit to height
            new_height = h
            new_width = int(h * photo_aspect_ratio)

        photo_resized = cv2.resize(photo, (new_width, new_height), interpolation=cv2.INTER_AREA)

        # Center the image in the cell if it does not match the exact cell size
        x_offset = x + (w - new_width) // 2
        y_offset = y + (h - new_height) // 2

        # Place the resized image into the output at the calculated position
        output_image[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = photo_resized

    return output_image

# Process each base image
for base_image_path in base_image_paths:
    # Load the base image
    base_image = cv2.imread(base_image_path)
    if base_image is None:
        print(f"Base image not found: {base_image_path}")
        continue

    # Detect grid cells
    cells = detect_grid_cells(base_image)
    if len(cells) == 0:
        print(f"No grid detected on base image {base_image_path}. Please check the image or adjust detection parameters.")
        continue

    # Overlay images on the detected grid cells
    output_image = overlay_images_on_grid(base_image_path, photo_paths, cells)
    
    # Save the output image
    base_image_name = os.path.basename(base_image_path).split('.')[0]
    output_path = os.path.join(output_folder, f"{base_image_name}-with-photos.jpg")
    cv2.imwrite(output_path, output_image)
    print(f"Saved output to: {output_path}")


Found 11 images in the folder 'D:\Vortex\ROV\Phase 2\Tasks2025\task 4\photos'.
Saved output to: D:\Vortex\ROV\Phase 2\Tasks2025\task 4\output\coral-reef-1-with-photos.jpg
Saved output to: D:\Vortex\ROV\Phase 2\Tasks2025\task 4\output\coral-reef-2-with-photos.jpg
